## Python Script that will combine CitiBike data for years 2014 to 2019.

#### Pre-Requisite to running the script
1. Download and extract all the required data files from [Citibike Data Page](https://www.citibikenyc.com/system-data) into a specific folder
2.provide the path of the folder in variable csvFilePath if it is not as given below

once above steps are completed, all set to run the program

In [7]:
# IMPORT DEPENDENCIES
import csv
import pandas as pd
import numpy as np
from datetime import datetime as dt
from dateutil import parser # used to convert string to date time without knowing the format
from zipfile import ZipFile as ZF
import os

In [2]:
#global variables
# csvFilePath variable : contains the filepath to the raw data files

csvFilePath = "rawdata"

gender = {0:"Unknown",1:"Male",2:"Female"} # to change gender values to meaningful information

In [89]:
# ************************** MAIN FUNCTION THAT READS CSV, CLEANS AND FORMATS CITIBIKE DATA FOR EACH YEAR *********************

def readYearlyData(csvFile):
    print(f"reading file ...............{csvFile}")
    # read citi bike data
    bikeDF = pd.read_csv(csvFile, low_memory=False, header =0, 
                         names = ["tripduration","starttime","stoptime","start station id","start station name",
                                    "start station latitude","start station longitude","end station id","end station name",
                                    "end station latitude","end station longitude","bikeid","usertype","birth year","gender"
])
    
    # CONVERT AND FORMAT FIELDS 
    
    #CHANGE DATE FIELDS FROM STRING TO DATE VALUE. DERIVE HOUR AND MDY INFORMATION AND STORE IT IN NEW COLUMNS
    bikeDF.starttime = bikeDF.starttime.apply(lambda d : parser.parse(d))
    
    bikeDF['start_Date'] = bikeDF.starttime.apply(lambda d: d.date())    
    bikeDF['start_hour'] = bikeDF.starttime.apply(lambda d: d.hour)
    
    # CONVERT TRIP DURATION FROM SECONDS TO HOURS
    bikeDF['tripduration_hrs'] = bikeDF.tripduration/3600
    
    # CONVERT GENDER TO MEANINGUL STRINGS
    bikeDF.gender = bikeDF.gender.apply(lambda r : gender[r])
    
    
    # DERIVE AGE OF THE CUSTOMER / SUBSCRIBER BASED ON BIRTH YEAR. IF AN INVALID BIRTH YEAR IS GIVEN, CATEGORIZE IT AS UNKNOWN
    if bikeDF['birth year'].dtype == 'object':
        bikeDF['birth year'] = bikeDF['birth year'].apply(lambda r: int(r) if r != "\\N" else 1800)
    else:
        bikeDF['birth year'].fillna(1800, inplace = True)
    
    bikeDF['age'] = bikeDF['birth year'].apply(lambda r : dt.now().year - r )

    # REPLACE ALL NaN VALUES TO APPROPRIATE VALUES
    bikeDF['start station id'].fillna(0, inplace = True)
    bikeDF['end station id'].fillna(0, inplace = True)
    bikeDF['start station name'].fillna("Unknown", inplace = True)
    bikeDF['end station name'].fillna("Unknown", inplace = True)

    # CONVERT RELEVANT VALUES TO INTEGER
    bikeDF['start station id'] = bikeDF['start station id'].astype(int)
    bikeDF['end station id'] = bikeDF['end station id'].astype(int)

    # GROUP AND BIN AGE AND HOURS TO OBSERVE PATTERN
    age_labels =["15-25","25-40","40-55","55-70","70+","Unknown"] 
    age_bins = [0,26,41,56, 71, 90, 250]
    bikeDF['AgeBins'] = pd.cut(bikeDF.age, bins = age_bins, labels = age_labels)

    hrs_labels = ["midnight-5AM","5AM - 9AM","10AM -1PM","1PM - 4PM","4PM - 7PM","7PM - midnight"]
    hrs_bins = [-1, 6, 10,14,17,20, 24]
    bikeDF['HrBins'] = pd.cut(bikeDF.start_hour, bins = hrs_bins, labels = hrs_labels)

    # ADD RIDES COLUMNS TO CALCULATE COUNT OF RIDES DURING GROUPING
    bikeDF['Rides'] = bikeDF.index

    # RETURN DEF
    return bikeDF

In [90]:
bikeAllYears = pd.DataFrame() # combined dataframe for Bike Data

In [91]:
# read files from zip file and concat data 
bikeAllYears = pd.concat(\
    [readYearlyData(os.path.join(csvFilePath, file)) for file in os.listdir(csvFilePath) \
     if file.endswith("csv")], \
        ignore_index = True)

reading file ...............rawdata\2014-02 - Citi Bike trip data.csv
reading file ...............rawdata\201502-citibike-tripdata.csv
reading file ...............rawdata\201602-citibike-tripdata.csv
reading file ...............rawdata\201702-citibike-tripdata.csv
reading file ...............rawdata\201802-citibike-tripdata.csv
reading file ...............rawdata\201902-citibike-tripdata.csv


In [93]:
bikeAllYears.count()

tripduration               3561045
starttime                  3561045
stoptime                   3561045
start station id           3561045
start station name         3561045
start station latitude     3561045
start station longitude    3561045
end station id             3561045
end station name           3561045
end station latitude       3561045
end station longitude      3561045
bikeid                     3561045
usertype                   3553465
birth year                 3561045
gender                     3561045
start_Date                 3561045
start_hour                 3561045
tripduration_hrs           3561045
age                        3561045
AgeBins                    3561045
HrBins                     3561045
Rides                      3561045
dtype: int64

In [94]:
# group bike data for visualization
bikeDF_grp = bikeAllYears[['start station id','start station name', 'end station id', 'end station name',
        'bikeid', 'usertype','gender', 'start_Date','tripduration_hrs', 'AgeBins','HrBins', 'Rides']]. \
        groupby(['start station id', 'start station name', 'end station id', 'end station name',
        'bikeid', 'usertype','gender', 'start_Date', 'AgeBins','HrBins']).\
        agg({'tripduration_hrs':'sum','Rides':'count'})

bikeDF_grp.reset_index(inplace = True)

In [95]:
bikeDF_grp.count()

start station id      3551256
start station name    3551256
end station id        3551256
end station name      3551256
bikeid                3551256
usertype              3551256
gender                3551256
start_Date            3551256
AgeBins               3551256
HrBins                3551256
tripduration_hrs      3551256
Rides                 3551256
dtype: int64

In [96]:
# save it to CSV
bikeDF_grp.to_csv("Allyears-citibike-tripdata-cleaned.csv", index = False)

In [97]:
bikeAllYears.columns

Index(['tripduration', 'starttime', 'stoptime', 'start station id',
       'start station name', 'start station latitude',
       'start station longitude', 'end station id', 'end station name',
       'end station latitude', 'end station longitude', 'bikeid', 'usertype',
       'birth year', 'gender', 'start_Date', 'start_hour', 'tripduration_hrs',
       'age', 'AgeBins', 'HrBins', 'Rides'],
      dtype='object')

In [98]:
# Derive year from given date
bikeAllYears['Year'] = bikeAllYears.start_Date.apply(lambda r : r.year)

In [99]:
# calculate path for caclulation of routes to and from station
bikeAllYears['Path_ID'] = bikeAllYears['start station id'].map(str)+ "__" + bikeAllYears['end station id'].map(str)+"__"\
                        + bikeAllYears['Year'].map(str)
bikeAllYears.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,...,gender,start_Date,start_hour,tripduration_hrs,age,AgeBins,HrBins,Rides,Year,Path_ID
0,382,2014-02-01 00:00:00,2014-02-01 00:06:22,294,Washington Square E,40.730494,-73.995721,265,Stanton St & Chrystie St,40.722293,...,Male,2014-02-01,0,0.106111,28.0,25-40,midnight-5AM,0,2014,294__265__2014
1,372,2014-02-01 00:00:03,2014-02-01 00:06:15,285,Broadway & E 14 St,40.734546,-73.990741,439,E 4 St & 2 Ave,40.726281,...,Female,2014-02-01,0,0.103333,40.0,25-40,midnight-5AM,1,2014,285__439__2014
2,591,2014-02-01 00:00:09,2014-02-01 00:10:00,247,Perry St & Bleecker St,40.735354,-74.004831,251,Mott St & Prince St,40.723180,...,Female,2014-02-01,0,0.164167,71.0,55-70,midnight-5AM,2,2014,247__251__2014
3,583,2014-02-01 00:00:32,2014-02-01 00:10:15,357,E 11 St & Broadway,40.732618,-73.991580,284,Greenwich Ave & 8 Ave,40.739017,...,Male,2014-02-01,0,0.161944,38.0,25-40,midnight-5AM,3,2014,357__284__2014
4,223,2014-02-01 00:00:41,2014-02-01 00:04:24,401,Allen St & Rivington St,40.720196,-73.989978,439,E 4 St & 2 Ave,40.726281,...,Male,2014-02-01,0,0.061944,29.0,25-40,midnight-5AM,4,2014,401__439__2014


In [100]:
# Obtain all origin station information
origin = bikeAllYears[['Path_ID','start station id','start station name', 'start station latitude',
       'start station longitude']]
origin.columns = ['Path_ID','Station_ID','Station_name', 'latitude',
       'longitude']
origin['cnt'] = origin.index
origin['Type'] = "begin"
origin.head()


E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Path_ID,Station_ID,Station_name,latitude,longitude,cnt,Type
0,294__265__2014,294,Washington Square E,40.730494,-73.995721,0,begin
1,285__439__2014,285,Broadway & E 14 St,40.734546,-73.990741,1,begin
2,247__251__2014,247,Perry St & Bleecker St,40.735354,-74.004831,2,begin
3,357__284__2014,357,E 11 St & Broadway,40.732618,-73.991580,3,begin
4,401__439__2014,401,Allen St & Rivington St,40.720196,-73.989978,4,begin


In [101]:
# Obtain Destination station informatio
dest = bikeAllYears[['Path_ID','end station id','end station name', 'end station latitude',
       'end station longitude']]
dest.columns = ['Path_ID','Station_ID','Station_name', 'latitude',
       'longitude']
dest['cnt'] = dest.index
dest['Type'] = "end"
dest.head()


E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Path_ID,Station_ID,Station_name,latitude,longitude,cnt,Type
0,294__265__2014,265,Stanton St & Chrystie St,40.722293,-73.991475,0,end
1,285__439__2014,439,E 4 St & 2 Ave,40.726281,-73.989780,1,end
2,247__251__2014,251,Mott St & Prince St,40.723180,-73.994800,2,end
3,357__284__2014,284,Greenwich Ave & 8 Ave,40.739017,-74.002638,3,end
4,401__439__2014,439,E 4 St & 2 Ave,40.726281,-73.989780,4,end


In [102]:
# combine both dataset to derive spatial dataset
spatial_data = origin.append(dest)

In [103]:
spatial_data.count()

Path_ID         7122090
Station_ID      7122090
Station_name    7122090
latitude        7122090
longitude       7122090
cnt             7122090
Type            7122090
dtype: int64

In [104]:
spatial_data_fnl = spatial_data.groupby(["Path_ID","Station_ID","Station_name","latitude","longitude","Type"]).agg("count")
spatial_data_fnl.reset_index(inplace = True)
spatial_data_fnl.count()

Path_ID         1029447
Station_ID      1029447
Station_name    1029447
latitude        1029447
longitude       1029447
Type            1029447
cnt             1029447
dtype: int64

In [105]:
# spatial_data_fnl = spatial_data_fnl[spatial_data_fnl.Path_ID != "0__0__2019"]
spatial_data_fnl.sort_values(['Path_ID','Type'], ascending = True, inplace = True)

spatial_data_fnl.to_csv("SpatialData_allyears.csv", index = False)

spatial_data_fnl.head()

,Path_ID,Station_ID,Station_name,latitude,longitude,Type,cnt
0,0__0__2019,0,Unknown,40.842,-73.881,begin,1
1,0__0__2019,0,Unknown,40.845,-73.902,begin,1
2,0__0__2019,0,Unknown,40.857,-73.890,begin,1
5,0__0__2019,0,Unknown,40.857,-73.881,begin,2
8,0__0__2019,0,Unknown,40.860,-73.884,begin,2


In [107]:
bikeAllYears.to_csv("AllYears_rawdata.csv", index = False)